In [ ]:
def hitrate_at_3(true_labels, pred_top3):
    hits = 0
    for true_labels, top3 in zip(true_labels, pred_top3):
        # есть ли пересечение между множеством истинных меток и предсказанных
        hits += int(any(lbl in top3 for lbl in true_labels))
    return hits / len(true_labels)

In [ ]:
# Пример кода - запускать не надо

import pandas as pd

# Читаем TSV/Parquet
apps = pd.read_csv("train.tsv", sep="\t")
#cars = pd.read_parquet("train_dataset.parquet")

# Клеим текст для задачи "Мыслим категориями"
apps["text"] = (
    apps["app_name"].fillna("") + " " +
    apps["shortDescription"].fillna("") + " " +
    apps["full_description"].fillna("")
).str.strip()

# Пример отправки (топ-3 метки уже посчитаны в pred_top3_str)
"""submit = pd.DataFrame({"app_name": test["app_name"], "labels_str": pred_top3_str})
submit.to_csv("submission.tsv", sep="\t", index=False)"""

'submit = pd.DataFrame({"app_name": test["app_name"], "labels_str": pred_top3_str})\nsubmit.to_csv("submission.tsv", sep="\t", index=False)'

In [ ]:
print(apps)

                                    app_name  \
0                        Матрона Московская.   
1        Run and Jump - "Бесконечный раннер"   
2                                 Ghost Maze   
3      LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ   
4                       Mario Anime Coloring   
...                                      ...   
53489                  Yota: мобильная связь   
53490                     BTS Musical Player   
53491                        Lite Calculator   
53492                               Optiland   
53493                       SelfiShop Camera   

                                        full_description  \
0      Хочешь знать что будет? Загляни в будущие. Мат...   
1      Run and Jump это новая увлекательная Аркада - ...   
2      Играя за приведение, собирай необходимые комби...   
3      AR-приложение «LabTools Mobile: Лабораторные п...   
4      \nРаскрасьте своих любимых персонажей из mario...   
...                                                  ...   
534

In [ ]:
import numpy as np

def median_ape(y_true, y_pred):
    # numpy-массивы будут удобнее
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    ape = np.abs(y_pred - y_true) / y_true
    return float(np.median(ape))

In [ ]:
!pip install  datasets

In [ ]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset('imdb')

# Посмотрим на пример данных
print(dataset['train'][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Загрузка токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Подготовка данных
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Преобразование данных в формат PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Создание DataLoader
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))  # Уменьшим выборку для примера
test_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'dataset' is not defined

In [ ]:
# Настройка оптимизатора и планировщика
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # 3 эпохи
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Перемещение модели на GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Функция для обучения
def train(model, train_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        # Перемещение данных на устройство (GPU/CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Прямой проход
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Train loss: {avg_loss}")

# Обучение модели
for epoch in range(3):
    print(f"Epoch {epoch + 1}")
    train(model, train_loader, optimizer, scheduler, device)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1
Train loss: 0.6963801121711731
Epoch 2
Train loss: 0.4313603132367134
Epoch 3
Train loss: 0.1880117779970169


In [ ]:
import numpy as np

# H@3: pred_probs - (n_samples, n_classes); true_sets - список множеств истинных меток
top3 = np.argsort(-pred_probs, axis=1)[:, :3]
h3 = np.mean([len(set(t) & set(row)) > 0 for t, row in zip(true_sets, top3)])

# medianAPE для «АвтоПрайс»
def median_ape(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.median(np.abs(y_pred - y_true) / y_true))

In [ ]:
# Функция для тестирования
def evaluate(model, test_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            # Перемещение данных на устройство (GPU/CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Прямой проход
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Вычисление точности
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total
    print(f"Test loss: {avg_loss}")
    print(f"Accuracy: {accuracy}")

# Оценка модели
evaluate(model, test_loader, device)

Test loss: 0.2845372482538223
Accuracy: 0.892


In [ ]:
"""
Updated training & inference script for .tsv dataset with columns:
- app_name
- full_description
- shortDescription
- labels_str (categories separated by '|')

Task: for each app in test set predict up to 3 categories ordered by confidence.

Usage examples:
python всерос_мл_1_updated.py --mode train --train_path data/train.tsv --val_size 0.1 --out_dir outputs
python всерос_мл_1_updated.py --mode predict --model_dir outputs --test_path data/test.tsv --out_pred submission.tsv

Requirements:
- Python 3.8+
- scikit-learn, pandas, joblib, tqdm

This script trains a TF-IDF + One-vs-Rest Logistic Regression classifier by default,
outputs model artifacts and a submission file with predicted categories joined by '|'.

"""

import os
import argparse
from typing import List, Tuple

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import joblib
from tqdm import tqdm


# ---------------------- Utilities ----------------------

def load_tsv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep='\t', dtype=str).fillna("")
    # Ensure expected columns exist
    expected = ['app_name', 'full_description', 'shortDescription', 'labels_str']
    for c in expected:
        if c not in df.columns:
            df[c] = ""
    return df


def combine_text(row) -> str:
    # Combine available textual fields into a single string
    parts = []
    if row.get('app_name'):
        parts.append(str(row['app_name']))
    if row.get('shortDescription'):
        parts.append(str(row['shortDescription']))
    if row.get('full_description'):
        parts.append(str(row['full_description']))
    return ' \n '.join([p for p in parts if p])


def prepare_labels(df: pd.DataFrame) -> Tuple[MultiLabelBinarizer, np.ndarray]:
    # Split labels_str by '|' into list. Empty -> []
    labels = df['labels_str'].fillna("").apply(lambda s: [x.strip() for x in s.split('|') if x.strip()])
    mlb = MultiLabelBinarizer(sparse_output=False)
    Y = mlb.fit_transform(labels)
    return mlb, Y


def prepare_texts(df: pd.DataFrame) -> List[str]:
    texts = df.apply(combine_text, axis=1)
    return texts.tolist()


# ---------------------- Model training & IO ----------------------

def train_tfidf_ovr(X_texts: List[str], Y: np.ndarray, out_dir: str, val_size: float = 0.1, random_state: int = 42):
    os.makedirs(out_dir, exist_ok=True)

    print('Vectorizing texts with TF-IDF...')
    vectorizer = TfidfVectorizer(
        max_features=100000,
        ngram_range=(1,2),
        min_df=3,
        max_df=0.9,
        strip_accents='unicode'
    )
    X = vectorizer.fit_transform(X_texts)
    print('Text shape:', X.shape)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=val_size, random_state=random_state)

    print('Training One-vs-Rest LogisticRegression...')
    # Use saga solver for multinomial; L2 penalty and class_weight='balanced' if imbalance
    base_clf = LogisticRegression(solver='saga', max_iter=2000, C=1.0)
    clf = OneVsRestClassifier(base_clf, n_jobs=-1)
    clf.fit(X_train, Y_train)

    print('Saving model artifacts...')
    joblib.dump(vectorizer, os.path.join(out_dir, 'tfidf_vectorizer.joblib'))
    joblib.dump(clf, os.path.join(out_dir, 'ovr_logreg.joblib'))

    # Save a quick validation report
    val_pred_prob = clf.predict_proba(X_val)
    # For top-1 eval convert to single-label using highest-prob per sample
    top1_preds = np.argmax(val_pred_prob, axis=1)
    # Convert argmax back to multilabel binary for F1 (only where class predicted)
    Y_top1 = np.zeros_like(Y_val)
    for i, c in enumerate(top1_preds):
        Y_top1[i, c] = 1

    micro_f1_top1 = f1_score(Y_val, Y_top1, average='micro', zero_division=0)
    print(f'Validation micro F1 (top-1): {micro_f1_top1:.4f}')

    # Evaluate top-3 using helper
    p_at_1, r_at_1, f1_at_1 = precision_recall_f1_at_k(Y_val, val_pred_prob, k=1)
    p_at_3, r_at_3, f1_at_3 = precision_recall_f1_at_k(Y_val, val_pred_prob, k=3)
    report = {
        'val_micro_f1_top1': float(micro_f1_top1),
        'p@1': float(p_at_1), 'r@1': float(r_at_1), 'f1@1': float(f1_at_1),
        'p@3': float(p_at_3), 'r@3': float(r_at_3), 'f1@3': float(f1_at_3),
    }
    joblib.dump(report, os.path.join(out_dir, 'validation_report.joblib'))
    print('Validation report:', report)

    return vectorizer, clf, report


# ---------------------- Prediction & evaluation helpers ----------------------

def predict_top_k(clf: OneVsRestClassifier, vectorizer: TfidfVectorizer, texts: List[str], k: int = 3) -> np.ndarray:
    X = vectorizer.transform(texts)
    probs = clf.predict_proba(X)
    n_classes = probs.shape[1]
    k_eff = min(k, n_classes)
    topk_indices = np.argsort(-probs, axis=1)[:, :k_eff]
    return topk_indices, probs


def indices_to_labels(topk_indices: np.ndarray, mlb: MultiLabelBinarizer) -> List[str]:
    label_names = mlb.classes_.tolist()
    out = []
    for row in topk_indices:
        labels = [label_names[i] for i in row]
        out.append('|'.join(labels))
    return out


def precision_recall_f1_at_k(Y_true: np.ndarray, pred_prob: np.ndarray, k: int = 3) -> Tuple[float, float, float]:
    # Y_true: binary matrix (n_samples, n_classes)
    n_samples, n_classes = Y_true.shape
    k_eff = min(k, n_classes)
    topk = np.argsort(-pred_prob, axis=1)[:, :k_eff]
    match_counts = 0
    total_true = Y_true.sum()
    for i in range(n_samples):
        pred_set = set(topk[i])
        true_set = set(np.where(Y_true[i] == 1)[0])
        match_counts += len(pred_set & true_set)
    precision_at_k = match_counts / (n_samples * k_eff)
    recall_at_k = match_counts / (total_true + 1e-12)
    if precision_at_k + recall_at_k == 0:
        f1 = 0.0
    else:
        f1 = 2 * precision_at_k * recall_at_k / (precision_at_k + recall_at_k)
    return precision_at_k, recall_at_k, f1


# ---------------------- Main CLI ----------------------

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--mode', choices=['train', 'predict'], required=True)
    parser.add_argument('--train_path', type=str, default='data/train.tsv')
    parser.add_argument('--test_path', type=str, default='data/test.tsv')
    parser.add_argument('--val_size', type=float, default=0.1)
    parser.add_argument('--out_dir', type=str, default='outputs')
    parser.add_argument('--model_dir', type=str, default='outputs')
    parser.add_argument('--out_pred', type=str, default='submission.tsv')
    parser.add_argument('--top_k', type=int, default=3)
    args = parser.parse_args()

    if args.mode == 'train':
        df = load_tsv(args.train_path)
        print('Loaded', len(df), 'rows from', args.train_path)
        texts = prepare_texts(df)
        mlb, Y = prepare_labels(df)
        # Save label classes
        os.makedirs(args.out_dir, exist_ok=True)
        joblib.dump(mlb, os.path.join(args.out_dir, 'mlb.joblib'))
        vectorizer, clf, report = train_tfidf_ovr(texts, Y, args.out_dir, val_size=args.val_size)
        print('Training finished. Artifacts saved to', args.out_dir)

    elif args.mode == 'predict':
        # Load artifacts
        mlb = joblib.load(os.path.join(args.model_dir, 'mlb.joblib'))
        vectorizer = joblib.load(os.path.join(args.model_dir, 'tfidf_vectorizer.joblib'))
        clf = joblib.load(os.path.join(args.model_dir, 'ovr_logreg.joblib'))

        df_test = load_tsv(args.test_path)
        texts = prepare_texts(df_test)
        topk_indices, probs = predict_top_k(clf, vectorizer, texts, k=args.top_k)
        pred_labels = indices_to_labels(topk_indices, mlb)

        # Attach predictions to dataframe and save
        out_df = df_test.copy()
        out_df['predicted_labels'] = pred_labels
        out_df.to_csv(args.out_pred, sep='\t', index=False)
        print('Predictions written to', args.out_pred)

        # If true labels exist in test, compute evaluation
        if 'labels_str' in df_test.columns and df_test['labels_str'].notnull().any():
            mlb_true = mlb
            labels_true = df_test['labels_str'].fillna("").apply(lambda s: [x.strip() for x in s.split('|') if x.strip()])
            Y_true = mlb_true.transform(labels_true)
            p_at_1, r_at_1, f1_at_1 = precision_recall_f1_at_k(Y_true, probs, k=1)
            p_at_3, r_at_3, f1_at_3 = precision_recall_f1_at_k(Y_true, probs, k=args.top_k)
            print(f'p@1={p_at_1:.4f}, r@1={r_at_1:.4f}, f1@1={f1_at_1:.4f}')
            print(f'p@{args.top_k}={p_at_3:.4f}, r@{args.top_k}={r_at_3:.4f}, f1@{args.top_k}={f1_at_3:.4f}')


if __name__ == '__main__':
    main()
